## Прогнозирование стоимости автомобиля по характеристикам

### Задача
-- создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам для того, чтобы выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены).

### Метрика 
MAPE (Mean Percentage Absolute Error) - средняя абсолютная ошибка в процентах

### Нужно
* Составить train датасет - спарсить данные, либо найти готовый
* Обучить модель

### Плюс
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Подгрузить еще больше данных
* Попробовать подобрать параметры модели
* Попробовать разные алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

### Этапы работы
* Парсинг с авто.ру - Нина
* EDA, Feature Engineering - Елена
* Сравнение одиночных моделей - Елена
* Стекинг

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns
import re
import sys 

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from tqdm.notebook import tqdm
from catboost import CatBoostRegressor

In [ ]:
pd.options.display.max_columns = None

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# Setup

In [ ]:
VERSION    = 17
VAL_SIZE   = 0.2   # 20%
RANDOM_SEED = 42

Загружаем **тестовый датасет** и шаблон **сабмишна**

In [ ]:
!ls ../input/

In [ ]:
!ls ../input/autoru-parsing

In [ ]:
train = pd.read_csv('../input/autoru-parsing/parsing.csv')
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [ ]:
train.head(5)

In [ ]:
train.info()

Признак `price` заполнен наполовину, возьмем только эту часть датасета

In [ ]:
train.dropna(axis=0, how='any', subset=['price'], inplace=True)

In [ ]:
train.columns

In [ ]:
train.info()

Отсутствуют данные в столбцах `image`, `modelDate`, `model_info`, `numberOfDoors`, `priceCurrency`, `super_gen`, `vehicleConfiguration`, `vendor`, `car_url` 

В `Владение` треть непустых значений

В остальном почти нет значительных пропусков

In [ ]:
test.head(5)

In [ ]:
test.info()

- `Train` - `32908` объектов
- `Test` - `34686` объектов

**Train** содержит данные по тем же бренадам, что и test:

In [ ]:
train.brand.unique()

In [ ]:
test.brand.unique()

## Data Preprocessing

In [ ]:
test['price'] = 0

Для корректной обработки признаков объединяем трейн и тест в один датасет

In [ ]:
columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower',
           'fuelType', 'modelDate', 'mileage', 'model_name', 'name', 'numberOfDoors',
           'productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'price']
df_train = train[columns]
df_test = test[columns]

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
df_train['sample'] = 1 # помечаем трейн
df_test['sample'] = 0 # помечаем тест

In [ ]:
data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data

In [ ]:
len(data)

Удалим дубликаты объявлений (по url) из трейна сохранив первое вхождение. 

Далее этот признак не понадобится - можно удалить.

In [ ]:
data.drop_duplicates(subset=['car_url'], keep='first', inplace=True, ignore_index=True)

In [ ]:
len(data)

In [ ]:
len(data[data['sample'] == 1])

Размер `train` сократился почти вдвое 

In [ ]:
data.drop(['car_url'], axis=1, inplace=True)

## EDA

['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower','fuelType', 'modelDate', 'mileage', 'model_name', 'name', 'numberOfDoors','productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', ]

### bodyType

In [ ]:
data['bodyType'].unique() 

In [ ]:
data['bodyType'].nunique() 

Попробуем выделить более крупные категории без инф-ции о кол-ве дверей и прочем, так как уже есть `numberOfDoors`

In [ ]:
pattern = re.compile('[а-я]*')
new_column = []
for string in data['bodyType']:
    new_column.append(pattern.match(str(string))[0])
data['bodyTypeShort'] = new_column

In [ ]:
data['bodyTypeShort'].nunique()

Число уникальных значений сократилось с 25 до 16. Далее можно будет оценить какой из признаков (старый или новый) сиьнее влияет на целевую переменную

### brand

In [ ]:
data['brand'].value_counts()

## color

In [ ]:
data['color'].value_counts()

## engineDisplacement

In [ ]:
data['engineDisplacement'].nunique()

In [ ]:
data['engineDisplacement'].unique()

Избавимся от 'LTR', преобразуем в вещественные числа и будем рассматривать признак как числовой

In [ ]:
data['engineDisplacement'].describe()

Медиана нам понадобится для замены пустого значения

In [ ]:
def to_float(column, pattern):
    new_column = []
    for string in column:
        if pattern.match(string) != None:
            new_column.append(float(pattern.match(string)[0]))
        else:
            new_column.append(2.0)
    return new_column

In [ ]:
pattern = re.compile('[0-9]\.[0-9]')
data['engineDisplacement'] = to_float(data['engineDisplacement'], pattern)

In [ ]:
data['engineDisplacement'].hist()

In [ ]:
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: np.log(x + 1))

In [ ]:
data['engineDisplacement'].hist()

## enginePower

In [ ]:
data['enginePower'].nunique()

In [ ]:
data['enginePower'].unique()

In [ ]:
data['enginePower'] = to_float(data['enginePower'], re.compile('[0-9]*'))

In [ ]:
data['enginePower'].hist()

## fuelType

In [ ]:
data['fuelType'].value_counts()

## modelDate

In [ ]:
data['modelDate'].hist()

In [ ]:
data['modelDate'].describe()

In [ ]:
data[data['modelDate'] >= 1980]['modelDate'].hist()

Выделить самые старые авто в отдельную категорию

## mileage

In [ ]:
data['mileage'].describe()

In [ ]:
data['mileage'].hist()

Попробовать выделить категории

## model_name

In [ ]:
data['model_name'].nunique()

Добавить новый признак - бренд + модель

In [ ]:
data['brand_model'] = data['brand'] + ' ' + data['model_name']
data['brand_model'].nunique()

## name

In [ ]:
data['name']

В трейне и тесте разные данные в столбце `name`, удалю его

In [ ]:
data.drop(['name'], axis=1, inplace=True)

## numberOfDoors

In [ ]:
data['numberOfDoors'].value_counts()

In [ ]:
data['numberOfDoors'][data['numberOfDoors'] == 0] = 3

In [ ]:
data['numberOfDoors'].value_counts()

## productionDate

In [ ]:
data['productionDate'].hist()

In [ ]:
data['productionDate'].describe()

In [ ]:
data[data['productionDate'] <= 1985]['productionDate'].value_counts()

до 1981 года авто можно объединить в одну группу

## vehicleTransmission

In [ ]:
data['vehicleTransmission'].value_counts()

## Владельцы

In [ ]:
data['Владельцы'].value_counts()

In [ ]:
pattern_owners = re.compile('[1-3]')
new_column = []
for string in data['Владельцы']:
    if pattern_owners.match(string) != None:
        new_column.append(int(pattern_owners.match(string)[0]))
data['Владельцы'] = new_column

In [ ]:
data['Владельцы'].value_counts()

## ПТС

Есть одно пустое значение - заполним модой

In [ ]:
data['ПТС'].value_counts()

In [ ]:
data['ПТС'] = data['ПТС'].fillna('Оригинал')

## Привод

In [ ]:
data['Привод'].value_counts()

## Руль

In [ ]:
data['Руль'].value_counts()

#### Сохраним список категориальных признаков в отдельную переменную

In [ ]:
cat_columns = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name', 'numberOfDoors',
           'productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'bodyTypeShort', 'brand_model']

In [ ]:
num_columns = ['engineDisplacement', 'enginePower', 'mileage', 'price']

#### Корреляция Пирсона

In [ ]:
sns.heatmap(data[num_columns].corr().abs(), vmin=0, vmax=1)

Высокая корреляция `engineDisplacement` с `enginePower`

Создать новый датасет с label_encoding, так как CatBoost лучше работает с некодированными признаками

In [ ]:
data

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price'], axis=1)
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)

In [ ]:
X

In [ ]:
y = data[data['sample'] == 1]['price']
# data.drop(['price'], axis=1, inplace=True)

In [ ]:
y

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Model 1: Наивная модель
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). C ней будем сравнивать другие модели.

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

## Model 2 : CatBoost

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         cat_features=cat_columns,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

### Log Traget
Попробуем взять таргет в логарифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).

В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в parsing_unixtime

In [ ]:
np.log(y_train)

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         cat_features=cat_columns,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         # plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

### Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

### CV

In [ ]:
# cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

In [ ]:
# cat_features_ids

In [ ]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = 3000,
                              learning_rate = 0.1,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_columns,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    
    return(model)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
X = X.reset_index(drop=True)

In [ ]:
y = y.reset_index(drop=True)

In [ ]:
submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=5,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

#### Submissions blend

In [ ]:
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = submissions['blend'].values
sample_submission.to_csv(f'submission_blend_v{VERSION}.csv', index=False)
sample_submission.head(10)

## Label Encoding

In [ ]:
def label_encoding(cat_columns):
    for column in cat_columns:
        data[column] = data[column].astype('category').cat.codes

In [ ]:
label_encoding(cat_columns)

In [ ]:
data

#### Оценка значимости категориальных признаков

In [ ]:
imp_cat = pd.Series(mutual_info_classif(data[data['price'] != 0][cat_columns], data[data['price'] != 0]['price'],
                                     discrete_features=True), index=cat_columns)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

### Linear Regression

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price', 'Руль', 'ПТС', 'numberOfDoors', 'fuelType'], axis=1)
X_sub = data.query('sample == 0').drop(['sample', 'price', 'Руль', 'ПТС', 'numberOfDoors', 'fuelType'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

С октября 2020 года цены выросли примерно на 8% (Росстат)

In [ ]:
y_train / 1.08

In [ ]:
# linreg = LinearRegression()
# linreg.fit(X_train, y_train/1.08)

In [ ]:
# predict_test = linreg.predict(X_test)
# predict_submission = linreg.predict(X_sub)

In [ ]:
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 93.57%

### DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(random_state=RANDOM_SEED)
model.fit(X_train, y_train/1.08)

In [ ]:
predict_test = model.predict(X_test)
predict_submission = model.predict(X_sub)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 20.05%

### RandomForestRegressor

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price'], axis=1)
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
rf = RandomForestRegressor(random_state=RANDOM_SEED)
rf.fit(X_train, y_train)
predict_test = rf.predict(X_test)
predict_submission = rf.predict(X_sub)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

`Точность модели по метрике MAPE: 14.91%`

In [ ]:
sns.barplot(rf.feature_importances_, X_train.columns)

In [ ]:
y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

In [ ]:
X = data.query('sample == 1')[['engineDisplacement', 'enginePower', 'fuelType', 'modelDate', 'mileage', 'model_name', 'productionDate', 'bodyTypeShort', 'brand_model']]
X_sub = data.query('sample == 0')[['engineDisplacement', 'enginePower', 'fuelType', 'modelDate', 'mileage', 'model_name', 'productionDate', 'bodyTypeShort', 'brand_model']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
rf = RandomForestRegressor(random_state=RANDOM_SEED)
rf.fit(X_train, y_train)
predict_test = rf.predict(X_test)
predict_submission = rf.predict(X_sub)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

## Boosting

In [ ]:
gbr = GradientBoostingRegressor(learning_rate=0.2, n_estimators=500, max_depth=5, min_samples_split=2, min_samples_leaf=2, subsample=1, random_state=RANDOM_SEED)
gbr.fit(X_train, y_train/1.08)
predict_test = gbr.predict(X_test)
predict_submission = gbr.predict(X_sub)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

`Точность модели по метрике MAPE: 15.01%`

## Stacking

Можно использовать `RandomForest`, `DecisionTree` и `CatBoost`(?) - они дали неплохие значения MAPE

In [ ]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):        
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

        X_meta_test += folded_clf.predict_proba(X_test)
    
    X_meta_test /= cv.n_splits
    return X_meta_train, X_meta_test   

In [ ]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [ ]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return f"Точность модели по метрике MAPE: {(mape(y_test, y_test_pred))*100:0.2f}%"

In [ ]:
# stacked_features_train, stacked_features_test = generate_meta_features([
#     RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=RANDOM_SEED),
#     LinearRegression(normalize=True, n_jobs=-1)
# ], X_train, X_test, y_train, cv)

In [ ]:
# compute_metric(LogisticRegression(penalty = 'none', multi_class = 'auto', solver='lbfgs', random_state = 42), stacked_features_train, y_train, stacked_features_test)

### Submission

In [ ]:
# predict_submission = model.predict(X_sub)
# predict_submission

In [ ]:
# sample_submission['price'] = predict_submission
# sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
# sample_submission.head(10)